# Ingesting Private Datasets (v1.4.1)

In [1]:
# imports
import warnings
warnings.filterwarnings('ignore')

import h5py
import specdb
import glob

from astropy.table import Table
from linetools import utils as ltu

from specdb.build import privatedb as pbuild

## Test on Single Folder

In [2]:
tree = specdb.__path__[0]+'/build/tests/files'
#os.getenv('DROPBOX_DIR')+'/QSOPairs/data/MMT_redux/'

In [3]:
reload(pbuild)
flux_files = pbuild.grab_files(tree)
len(flux_files)

2

In [4]:
flux_files[:5]

([u'/Users/xavier/local/Python/specdb/specdb/build/tests/files//./SDSSJ001605.89+005654.3_b800_F.fits.gz',
  u'/Users/xavier/local/Python/specdb/specdb/build/tests/files//./SDSSJ001607.27+005653.1_b800_F.fits.gz'],
 None)

----

## Directory Tree

In [9]:
tree2 = specdb.__path__[0]+'/data/test_privateDB'

In [10]:
branches = glob.glob(tree2+'/*')

### Files

In [12]:
reload(pbuild)
mflux_files, meta_file = pbuild.grab_files(branches[0])
len(mflux_files)

2

In [13]:
mflux_files[:5]

[u'/Users/xavier/local/Python/specdb/specdb/data/test_privateDB/COS//./J095240.17+515250.03.fits.gz',
 u'/Users/xavier/local/Python/specdb/specdb/data/test_privateDB/COS//./J095243.05+515121.15.fits.gz']

In [14]:
meta_file

u'/Users/xavier/local/Python/specdb/specdb/data/test_privateDB/COS/COS_meta.json'

In [15]:
meta_dict = ltu.loadjson(meta_file)
meta_dict

{u'maxpix': 60000,
 u'meta_dict': {u'TELESCOPE': u'HST'},
 u'parse_head': {u'DATE-OBS': u'DATE',
  u'GRATING': u'OPT_ELEM',
  u'INSTR': u'INSTRUME',
  u'R': True}}

### ztbl  (read from file)

In [17]:
ztbl = Table.read(specdb.__path__[0]+'/data/test_privateDB/testDB_ztbl.fits')

### Meta

In [18]:
reload(pbuild)
meta = pbuild.mk_meta(mflux_files, ztbl, fname=True, mdict=meta_dict['meta_dict'], parse_head=meta_dict['parse_head'])

In [19]:
meta[0:3]

RA_SPEC,DEC_SPEC,STYPE,zem,sig_zem,flag_zem,SPEC_FILE,DATE-OBS,GRATING,R,INSTR,TELESCOPE,EPOCH,GROUP_ID
float64,float64,str3,float64,float64,str8,unicode96,str10,str5,float64,str3,unicode3,float64,int64
148.167375,51.8805638889,QSO,1.3,0.0,UNKNW,/Users/xavier/local/Python/specdb/specdb/data/test_privateDB/COS//./J095240.17+515250.03.fits.gz,2015-05-31,G130M,17000.0,COS,HST,2000.0,0
148.179375,51.855875,QSO,1.4,0.0,UNKNW,/Users/xavier/local/Python/specdb/specdb/data/test_privateDB/COS//./J095243.05+515121.15.fits.gz,2015-12-08,G130M,17000.0,COS,HST,2000.0,1


### Spectra

In [20]:
hdf = h5py.File('tmp.hdf5','w')

In [21]:
reload(pbuild)
pbuild.ingest_spectra(hdf, 'test', meta, max_npix=meta_dict['maxpix'])

Adding test group to DB


In [22]:
hdf.close()

## All in One

In [23]:
reload(pbuild)
pbuild.mk_db(tree2, 'test', 'tmp.hdf5', ztbl, fname=True)

Wrote tmp.hdf5 DB file


----

## JSON files for meta table

In [25]:
parse_head = {'DATE-OBS':'DATE', 'TELESCOPE':'TELESCOP','INSTR':'INSTRUME', 'R': True}
mdict = dict(GRATING='ALL', R=8000.)

In [26]:
db_dict = dict(parse_head=parse_head, meta_dict=mdict, maxpix=60000)

In [27]:
jdict = ltu.jsonify(db_dict)

In [28]:
ltu.savejson('tst.json', jdict, easy_to_read=True)